This is a markdown cell. It describes the purpose of this notebook and its code.The other cells are coding cells. 

This notebook will transform a PDF scan of a Latin record into Word doc (Latin), and then translate it to English including its numerals. Final output is a Word doc (English).

Last updated by Kuba Kowalski on 11/11/2025, 23:00.

## Applying OCR to raw scans

Embedding selectable text into PDF to make conversion to Word possible. Note, this is needed because while the original scans appear to have selectable text, it's actually full of errors and cannot be trusted. 

OCR is the standard used for other Winchester Pipe Rolls. No need to reinvent the wheel. 

In [2]:
# Install for convenience
!pip install ocrmypdf
!sudo apt-get install -y tesseract-ocr tesseract-ocr-lat


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
Sudo is disabled on this machine. To enable it, go to the ]8;;ms-settings:developers\Developer Settings page]8;;\ in the Settings app


In [ ]:
# Import packages
import os
import subprocess
from pathlib import Path

In [ ]:
# Example bash !ocrmypdf --force-ocr -l lat+eng Cuxham_1288.pdf Cuxham_1288_ocr.pdf
    ## --force-ocr is needed to overwrite original text layer
    ## -l lat+eng is needed to handle mix of English & Latin, suggested by GPT-5

# Paths

## Input
input_dir = Path(r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\input\Latin\Cuxham")

## Output: where to save OCR’d files (can be same as input)
output_dir = Path(r"C:\Users\kubak\OneDrive - Wageningen University & Research\WUR\2024-2025\research_assistant_ox\workfolder\output\Latin\Cuxham")
output_dir.mkdir(exist_ok=True)

# The bash command above is rebuilt for Python use below
    ## Most convenient way for looping through multiple input PDFs (Latin)
for pdf_file in input_dir.glob("*.pdf"):
    output_pdf = output_dir / f"{pdf_file.stem}_ocr.pdf"
    
    print(f"OCR processing: {pdf_file.name} -> {output_pdf.name}")

    cmd = [
        "ocrmypdf",
        "--force-ocr",
        "-l", "lat+eng",
        str(pdf_file),
        str(output_pdf)
    ]

    try:
        subprocess.run(cmd, check=True)
        print(f"Task succesful: {pdf_file.name}")
    except subprocess.CalledProcessError as e:
        print(f"Task failed (fix your directories): {pdf_file.name}: {e}")